In [14]:
!pip install scikit-image
!pip install image-quality
!pip install mahotas
!pip install lpips
!pip install pytorch-msssim

In [15]:
# SSIM
from skimage.metrics import structural_similarity as ssim
import cv2

# GMSD
import imquality.brisque as brisque
import numpy as np
import pandas as pd
import mahotas

# LPIPS
import torch
import lpips
from PIL import Image
lpips_model = lpips.LPIPS(net='alex')  # can also use 'vgg' or 'squeeze'

# TFS
from scipy.stats import pearsonr

# MS-SSIM
from torchvision import transforms
from pytorch_msssim import ms_ssim

from google.colab.patches import cv2_imshow

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


In [16]:
######################################################
######### LOW LEVEL METRICS ##########################
######################################################

# SSIM (Structural Similarity Index Measure)
def calculate_ssim(img1, img2):
    """
    [0, 1]
    0: identical
    1: completely different
    """
    # Compute SSIM
    score, _ = ssim(img1, img2, full=True)
    return score

# GMSD (Gradient Magnitude Similarity Deviation)
def calculate_gmsd(img1, img2):
    """
    [0, ∞)
    0: identical
    ∞: completely different
    """
    # Calculate gradients
    g1 = cv2.Laplacian(img1, cv2.CV_64F)
    g2 = cv2.Laplacian(img2, cv2.CV_64F)
    # Calculate the GMSD
    gmsd_value = np.std(g1 - g2)
    return gmsd_value

# NLPD (Normalized Laplacian Pyramid Distance)
def calculate_nlpd(img1, img2):
    """
    [0, ∞)
    0: identical
    ∞: completely different
    """
    # Downsample and compute Laplacian pyramids
    laplacian_pyramid_img1 = cv2.pyrDown(cv2.Laplacian(img1, cv2.CV_64F))
    laplacian_pyramid_img2 = cv2.pyrDown(cv2.Laplacian(img2, cv2.CV_64F))

    # Calculate normalized distance
    nlpd_value = np.mean(np.abs(laplacian_pyramid_img1 - laplacian_pyramid_img2))
    return nlpd_value

In [17]:
######################################################
######### HIGH LEVEL METRICS ##########################
######################################################


# Learned Perceptual Image Patch Similarity (LPIPS)
def calculate_lpips(img1, img2):
    """
    [0, 1]
    0 indicates perfect similarity
    1 indicates maximum difference
    """
    # If img1 and img2 are numpy arrays, convert them to PIL images
    if isinstance(img1, np.ndarray):
        img1 = Image.fromarray(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    if isinstance(img2, np.ndarray):
        img2 = Image.fromarray(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))

    # Resize images to the same size if they differ
    if img1.size != img2.size:
        img2 = img2.resize(img1.size, Image.BILINEAR)

    # Convert images to tensors and normalize to LPIPS input format
    img1_tensor = lpips.im2tensor(np.array(img1))  # Convert image to normalized tensor
    img2_tensor = lpips.im2tensor(np.array(img2))

    # Calculate LPIPS distance
    lpips_distance = lpips_model(img1_tensor, img2_tensor)

    return lpips_distance.item()


# Texture Frequency Spectrum (Fourier Transform-based)
# TODO Suspicious results ??????
def texture_frequency_similarity(img1, img2):
    """
    [-1,1]
    -1: perfectly inversely related
    0: no correlation
    1: identical
    """
    def calculate_fourier_magnitude(image):
        # Compute Fourier Transform
        f_transform = np.fft.fft2(image)
        # Shift the zero-frequency component to the center of the spectrum
        f_transform_shifted = np.fft.fftshift(f_transform)
        # Compute the magnitude spectrum
        magnitude_spectrum = np.abs(f_transform_shifted)
        return magnitude_spectrum

    # Calculate the Fourier magnitude spectra of both images
    mag_spectrum1 = calculate_fourier_magnitude(img1)
    mag_spectrum2 = calculate_fourier_magnitude(img2)

    # Flatten the spectra to 1D for comparison
    mag_spectrum1_flat = mag_spectrum1.flatten()
    mag_spectrum2_flat = mag_spectrum2.flatten()

    # Calculate similarity using Pearson correlation coefficient
    # (Values closer to 1 indicate higher similarity)
    similarity, _ = pearsonr(mag_spectrum1_flat, mag_spectrum2_flat)
    return similarity

# Earth Mover's Distance (EMD)
def calculate_emd(img1, img2):
    """
    [0, ∞)
    0 indicates perfect similarity
    Higher values indicate greater dissimilarity
    """
    def calculate_grayscale_histogram(image, bins=256):
        """
        Calculates and normalizes the grayscale histogram for an image.
        """
        # Calculate histogram
        hist = cv2.calcHist([image], [0], None, [bins], [0, 256])
        # Flatten the histogram and normalize it to have a sum of 1
        hist = cv2.normalize(hist, hist).flatten()
        return hist

    def calculate(hist1, hist2):
        """
        Calculates Earth Mover's Distance (EMD) between two histograms.
        The histograms are converted to a suitable format for OpenCV's EMD function.
        """
        # Reshape histograms to single-column for EMD compatibility
        signature1 = np.array([[h, float(i)] for i, h in enumerate(hist1)], dtype=np.float32)
        signature2 = np.array([[h, float(i)] for i, h in enumerate(hist2)], dtype=np.float32)

        # Calculate EMD
        emd_value, _, _ = cv2.EMD(signature1, signature2, cv2.DIST_L2)
        return emd_value

    hist1 = calculate_grayscale_histogram(img1)
    hist2 = calculate_grayscale_histogram(img2)
    emd_value = calculate(hist1, hist2)
    return emd_value

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
###### IMAGE PREPROCESSING #######

# Convert image to grayscale only if it has more than one channel
def convert_grayscale(img):
    if len(img.shape) == 3:  # Check if the image has multiple channels
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = img  # Already grayscale
    return gray_image

# Preprocess images for comparison
def preprocess_image(img1, img2):
    # Convert to grayscale
    gray_img1 = convert_grayscale(img1)
    gray_img2 = convert_grayscale(img2)

    # Ensure both images have the same dimensions
    if gray_img1.shape != gray_img2.shape:
        min_height = min(gray_img1.shape[0], gray_img2.shape[0])
        min_width = min(gray_img1.shape[1], gray_img2.shape[1])

        # Crop both images from the center to ensure they match
        def crop_center(img, height, width):
            start_y = (img.shape[0] - height) // 2
            start_x = (img.shape[1] - width) // 2
            return img[start_y:start_y + height, start_x:start_x + width]

        gray_img1 = crop_center(gray_img1, min_height, min_width)
        gray_img2 = crop_center(gray_img2, min_height, min_width)

    return gray_img1, gray_img2

In [ ]:
#####################################
###### PLOT METRIC DISTRIBUTION #####
#####################################

In [20]:
####### LOAD REAL & SYNTHETIC IMAGES ########

import os

# Path to the main folder (update this to your folder path in Google Drive)
synthetic_image_folder = "/content/drive/My Drive/mars_images/test_images"
real_image_folder = "/content/drive/My Drive/mars_images/real_images"


def load_images_from_folder(image_folder):

  # Dictionary to store images by class
  images_by_class = {}

  # Iterate through the subfolders
  for class_folder in os.listdir(image_folder):
      class_path = os.path.join(image_folder, class_folder)
      if os.path.isdir(class_path):  # Ensure it's a folder
          images_by_class[class_folder] = []
          for image_file in os.listdir(class_path):
              if image_file.lower().endswith((".jpg", ".png", ".jpeg", ".bmp")):  # Check for image files
                  images_by_class[class_folder].append(os.path.join(class_path, image_file))
  return images_by_class

synth_images_by_class = load_images_from_folder(synthetic_image_folder)
real_images_by_class = load_images_from_folder(real_image_folder)

# Print summary of images by class
for class_name, image_list in synth_images_by_class.items():
    print(f"Synthetic Class: {class_name}, Number of Images: {len(image_list)}")
    # print(f"Sample Images: {image_list[:3]}")  # Show a few sample images for verification

for class_name, image_list in real_images_by_class.items():
    print(f"Real Class: {class_name}, Number of Images: {len(image_list)}")
    # print(f"Sample Images: {image_list[:3]}")  # Show a few sample images for verification

Synthetic Class: cliffs, Number of Images: 54
Synthetic Class: craters, Number of Images: 161
Synthetic Class: subsurface_cave, Number of Images: 78
Synthetic Class: lava_tubes, Number of Images: 23
Synthetic Class: volcanic_plains_lava_flows, Number of Images: 76
Synthetic Class: layer_sediments, Number of Images: 82
Synthetic Class: rock_fields, Number of Images: 120
Synthetic Class: sand_dunes, Number of Images: 48
Synthetic Class: ridges, Number of Images: 115
Real Class: cliffs, Number of Images: 5
Real Class: craters, Number of Images: 5
Real Class: layer_sediments, Number of Images: 5
Real Class: ridges, Number of Images: 5
Real Class: rock_fields, Number of Images: 5
Real Class: sand_dunes, Number of Images: 5
Real Class: subsurface_cave, Number of Images: 5
Real Class: volcanic_plains_lava_flows, Number of Images: 5


In [21]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2  # Ensure OpenCV is imported

def synthetic_images_diversity_plots(image_list, metric_functions, class_name):
    """
    Perform pairwise analysis for a list of images using specified metric functions and save plots.

    Parameters:
    - image_list: List of file paths to images.
    - metric_functions: List of metric functions. Each function should take two images as input
                        and return a similarity score.
    - class_name: Name of the class (e.g., 'cliffs') used for naming the output file.

    Returns:
    - None (Saves the distribution plots of each metric in a single image file).
    """
    # Dictionary to store results for each metric
    metric_results = {metric_func.__name__: [] for metric_func in metric_functions}

    # Pairwise comparisons
    for img1, img2 in tqdm(itertools.combinations(image_list, 2), desc="Pairwise Comparisons"):

        # Load the images
        image1 = cv2.imread(img1, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        image2 = cv2.imread(img2, cv2.IMREAD_GRAYSCALE)

        # Preprocess
        image1, image2 = preprocess_image(image1, image2)

        # Calculate each metric
        for metric_func in metric_functions:
            try:
                score = metric_func(image1, image2)
                metric_results[metric_func.__name__].append(score)
            except Exception as e:
                print(f"Error calculating {metric_func.__name__} for a pair: {e}")
                continue

    # Save plots into a single file
    num_metrics = len(metric_results)
    fig, axes = plt.subplots(num_metrics, 1, figsize=(10, 6 * num_metrics))

    for i, (metric_name, scores) in enumerate(metric_results.items()):
        ax = axes[i] if num_metrics > 1 else axes  # Handle single plot case
        ax.hist(scores, bins=30, alpha=0.7, color='blue', density=True)
        ax.set_title(f"Distribution of {metric_name}")
        ax.set_xlabel(f"{metric_name} Score")
        ax.set_ylabel("Density")
        ax.grid(True)

    # Save the figure
    output_filename = f"{class_name}_synth_diversity.png"
    plt.tight_layout()
    plt.savefig(output_filename)
    print(f"Plots saved to {output_filename}")
    plt.close(fig)

    return metric_results

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import os

def metric_sensitivity(real_images, synthetic_image_paths, metric_functions, synth_class_name, real_class_name, drive_path):
    """
    Compare synthetic images from one class to real images of another class using specified metrics over multiple rounds.

    Parameters:
    - real_images: List of file paths to real images for each round.
    - synthetic_image_paths: List of file paths to synthetic images.
    - metric_functions: List of metric functions. Each function should take two images as input
                        and return a similarity score.
    - synth_class_name: Name of the synthetic images' class (e.g., 'cliffs').
    - real_class_name: Name of the real images' class (e.g., 'ridges') used for naming the output file.
    - drive_path: Path to the Google Drive folder where the output image will be saved.

    Returns:
    - None (Saves the combined plots as a single image).
    """
    num_rounds = len(real_images)
    num_metrics = len(metric_functions)

    # Initialize results container
    results = []

    for round_idx, real_image_path in enumerate(real_images, start=1):
        print(f"Processing Round {round_idx}/{num_rounds} comparing synthetic {synth_class_name} to real {real_class_name}...")
        # Load the real image
        real_image = cv2.imread(real_image_path, cv2.IMREAD_COLOR)

        # Ensure the real image is loaded
        if real_image is None:
            print(f"Error: Could not load real image at path: {real_image_path}")
            continue

        # Dictionary to store results for the current round
        round_results = {metric_func.__name__: [] for metric_func in metric_functions}

        # Compare the real image with each synthetic image
        for synthetic_image_path in tqdm(synthetic_image_paths, desc=f"Round {round_idx}"):
            # Load synthetic image
            synthetic_image = cv2.imread(synthetic_image_path, cv2.IMREAD_COLOR)

            # Ensure the synthetic image is loaded
            if synthetic_image is None:
                print(f"Error: Could not load synthetic image at path: {synthetic_image_path}")
                continue

            # Preprocess images
            real_preprocessed, synthetic_preprocessed = preprocess_image(real_image, synthetic_image)

            # Calculate metrics
            for metric_func in metric_functions:
                try:
                    score = metric_func(real_preprocessed, synthetic_preprocessed)
                    round_results[metric_func.__name__].append(score)
                except Exception as e:
                    print(f"Error calculating {metric_func.__name__}: {e}")

        results.append(round_results)

    # Create a combined plot
    fig, axes = plt.subplots(num_rounds, num_metrics, figsize=(5 * num_metrics, 4 * num_rounds))
    fig.suptitle(f"Metric Sensitivity Analysis: {synth_class_name} vs {real_class_name}", fontsize=16)

    for round_idx, round_results in enumerate(results):
        for metric_idx, (metric_name, scores) in enumerate(round_results.items()):
            if num_rounds > 1 and num_metrics > 1:
                ax = axes[round_idx, metric_idx]
            elif num_rounds > 1:
                ax = axes[round_idx]
            elif num_metrics > 1:
                ax = axes[metric_idx]
            else:
                ax = axes

            ax.hist(scores, bins=30, alpha=0.7, color='pink', density=True)
            ax.set_title(f"{metric_name} (Round {round_idx + 1})")
            ax.set_xlabel("Score")
            ax.set_ylabel("Density")
            ax.grid(True)

    # Save the combined plot to Google Drive
    os.makedirs(drive_path, exist_ok=True)  # Ensure the directory exists
    output_filename = os.path.join(drive_path, f"synth_{synth_class_name}_vs_real_{real_class_name}_metric_sensitivity.png")
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave space for the title
    plt.savefig(output_filename)
    print(f"Combined plots saved to {output_filename}")
    plt.close(fig)

In [23]:
classes = ['cliffs', 'ridges', 'layer_sediments', 'rock_fields', 'craters', 'volcanic_plains_lava_flows', 'sand_dunes', 'subsurface_cave']

# selected metrics per class to evaluate on
cliffs_metrics = [calculate_ssim, calculate_gmsd, calculate_nlpd, calculate_lpips]
ridges_metrics = [calculate_ssim, calculate_gmsd, calculate_nlpd, calculate_lpips, texture_frequency_similarity]
layer_metrics = [calculate_ssim, calculate_gmsd, calculate_nlpd, calculate_lpips, calculate_emd]
rock_metrics = [calculate_ssim, calculate_gmsd, calculate_nlpd, calculate_lpips, calculate_emd]
craters_metrics = [calculate_ssim, calculate_gmsd, calculate_nlpd, calculate_lpips, calculate_emd]
lava_flow_metrics = [calculate_ssim, calculate_gmsd, calculate_nlpd, calculate_lpips, calculate_emd]
sand_dunes_metrics = [calculate_ssim, calculate_gmsd, calculate_nlpd, calculate_lpips, calculate_emd]
subsurface_cavities_metrics = [calculate_ssim, calculate_gmsd, calculate_nlpd, calculate_lpips]

metrics_by_class = {
    'cliffs': cliffs_metrics,
    'ridges': ridges_metrics,
    'layer_sediments': layer_metrics,
    'rock_fields': rock_metrics,
    'craters': craters_metrics,
    'volcanic_plains_lava_flows': lava_flow_metrics,
    'sand_dunes': sand_dunes_metrics,
    'subsurface_cave': subsurface_cavities_metrics
}

In [ ]:
###############################################
###### SYNTHETIC DATA DIVERSITY ANALYSIS ######
##############################################

# synthetic image diversity (DONE)
cliffs_results = synthetic_images_diversity_plots(synth_images_by_class['cliffs'], cliffs_metrics, 'cliffs')
ridges_results = synthetic_images_diversity_plots(synth_images_by_class['ridges'], ridges_metrics, 'ridges')
layer_results = synthetic_images_diversity_plots(synth_images_by_class['layer_sediments'], layer_metrics, 'layer_sediments')
rock_results = synthetic_images_diversity_plots(synth_images_by_class['rock_fields'], rock_metrics, 'rock_fields')
craters_results = synthetic_images_diversity_plots(synth_images_by_class['craters'], craters_metrics, 'craters')
lava_flow_results = synthetic_images_diversity_plots(synth_images_by_class['volcanic_plains_lava_flows'], lava_flow_metrics, 'volcanic_plains_lava_flows')

In [24]:
# synthetic image diversity TODO
sand_dunes_results = synthetic_images_diversity_plots(synth_images_by_class['sand_dunes'], sand_dunes_metrics, 'sand_dunes')
subsurface_cavities_results = synthetic_images_diversity_plots(synth_images_by_class['subsurface_cave'], subsurface_cavities_metrics, 'subsurface_cave')

Pairwise Comparisons: 1128it [17:07,  1.10it/s]


Plots saved to sand_dunes_synth_diversity.png


Pairwise Comparisons: 3003it [22:52,  2.19it/s]


Plots saved to subsurface_cave_synth_diversity.png


In [13]:
###############################################
###### METRIC SENSITIVITY ANALYSIS ######
##############################################

# Loop through all combinations of real and synthetic classes
for real_class in classes:
    for synth_class in classes:

        print(f"Evaluating synthetic class '{synth_class}' against real class '{real_class}'...")

        # Get the metrics specific to the real class
        metrics = metrics_by_class.get(real_class, [])

        # Get real and synthetic images for the respective classes
        real_images = real_images_by_class.get(real_class, [])
        synth_images = synth_images_by_class.get(synth_class, [])

        # Perform metric sensitivity evaluation
        if real_images and synth_images:
            metric_sensitivity(
                real_images=real_images,
                synthetic_image_paths=synth_images,
                metric_functions=metrics,
                synth_class_name=synth_class,
                real_class_name=real_class,
                drive_path='/content/drive/My Drive/mars_images'
            )
        else:
            print(f"Skipping evaluation for '{synth_class}' vs '{real_class}' due to missing images.")

Evaluating synthetic class 'cliffs' against real class 'cliffs'...
Processing Round 1/5 comparing synthetic cliffs to real cliffs...


Round 1: 100%|██████████| 54/54 [00:21<00:00,  2.54it/s]


Processing Round 2/5 comparing synthetic cliffs to real cliffs...


Round 2: 100%|██████████| 54/54 [00:20<00:00,  2.61it/s]


Processing Round 3/5 comparing synthetic cliffs to real cliffs...


...

Processing Round 4/5 comparing synthetic subsurface_cave to real subsurface_cave...


Round 4: 100%|██████████| 78/78 [00:32<00:00,  2.37it/s]


Processing Round 5/5 comparing synthetic subsurface_cave to real subsurface_cave...


Round 5: 100%|██████████| 78/78 [00:33<00:00,  2.36it/s]


Combined plots saved to /content/drive/My Drive/mars_images/synth_subsurface_cave_vs_real_subsurface_cave_metric_sensitivity.png


In [27]:
# # real lava flow vs. sand dunes + subsurface cavities
# for real_class in ['volcanic_plains_lava_flows']:
#   for synth_class in ['sand_dunes', 'subsurface_cave']:
#       print(f"Evaluating synthetic class '{synth_class}' against real class '{real_class}'...")

#       # Get the metrics specific to the real class
#       metrics = metrics_by_class.get(real_class, [])

#       # Get real and synthetic images for the respective classes
#       real_images = real_images_by_class.get(real_class, [])
#       synth_images = synth_images_by_class.get(synth_class, [])

#       # Perform metric sensitivity evaluation
#       if real_images and synth_images:
#           metric_sensitivity(
#               real_images=real_images,
#               synthetic_image_paths=synth_images,
#               metric_functions=metrics,
#               synth_class_name=synth_class,
#               real_class_name=real_class,
#               drive_path='/content'
#           )
#       else:
#           print(f"Skipping evaluation for '{synth_class}' vs '{real_class}' due to missing images.")

Evaluating synthetic class 'sand_dunes' against real class 'volcanic_plains_lava_flows'...
Processing Round 1/5 comparing synthetic sand_dunes to real volcanic_plains_lava_flows...


Round 1: 100%|██████████| 48/48 [00:29<00:00,  1.63it/s]


Processing Round 2/5 comparing synthetic sand_dunes to real volcanic_plains_lava_flows...


Round 2: 100%|██████████| 48/48 [00:18<00:00,  2.57it/s]


Processing Round 3/5 comparing synthetic sand_dunes to real volcanic_plains_lava_flows...


...

Processing Round 4/5 comparing synthetic subsurface_cave to real volcanic_plains_lava_flows...


Round 4: 100%|██████████| 78/78 [00:29<00:00,  2.65it/s]


Processing Round 5/5 comparing synthetic subsurface_cave to real volcanic_plains_lava_flows...


Round 5: 100%|██████████| 78/78 [00:43<00:00,  1.81it/s]


Combined plots saved to /content/synth_subsurface_cave_vs_real_volcanic_plains_lava_flows_metric_sensitivity.png


In [29]:
# # real sand dunes + all the other synthetic image classes
# for real_class in ['sand_dunes', 'subsurface_cave']:
#   for synth_class in classes:
#       print(f"Evaluating synthetic class '{synth_class}' against real class '{real_class}'...")

#       # Get the metrics specific to the real class
#       metrics = metrics_by_class.get(real_class, [])

#       # Get real and synthetic images for the respective classes
#       real_images = real_images_by_class.get(real_class, [])
#       synth_images = synth_images_by_class.get(synth_class, [])

#       # Perform metric sensitivity evaluation
#       if real_images and synth_images:
#           metric_sensitivity(
#               real_images=real_images,
#               synthetic_image_paths=synth_images,
#               metric_functions=metrics,
#               synth_class_name=synth_class,
#               real_class_name=real_class,
#               drive_path='/content'
#           )
#       else:
#           print(f"Skipping evaluation for '{synth_class}' vs '{real_class}' due to missing images.")

Evaluating synthetic class 'cliffs' against real class 'sand_dunes'...
Processing Round 1/5 comparing synthetic cliffs to real sand_dunes...


Round 1: 100%|██████████| 54/54 [00:25<00:00,  2.10it/s]


Processing Round 2/5 comparing synthetic cliffs to real sand_dunes...


Round 2: 100%|██████████| 54/54 [00:27<00:00,  1.93it/s]


Processing Round 3/5 comparing synthetic cliffs to real sand_dunes...


...

Processing Round 4/5 comparing synthetic subsurface_cave to real subsurface_cave...


Round 4: 100%|██████████| 78/78 [00:29<00:00,  2.61it/s]


Processing Round 5/5 comparing synthetic subsurface_cave to real subsurface_cave...


Round 5: 100%|██████████| 78/78 [00:29<00:00,  2.62it/s]


Combined plots saved to /content/synth_subsurface_cave_vs_real_subsurface_cave_metric_sensitivity.png
